<a href="https://colab.research.google.com/github/OVP2023/CV/blob/main/tracking_%D0%BB%D0%B8%D1%86%D0%B0_%D1%81_%D0%B2%D0%B5%D0%B1_%D0%BA%D0%B0%D0%BC%D0%B5%D1%80%D1%8B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Пример работы трекера лица по видео с веб-камеры
import cv2

print(cv2.__version__)

4.11.0


In [ ]:
#Загружаем предобученную модель детектора лица
face_cascade = cv2.CascadeClassifier('/content/drive/MyDrive/Datasets/CV/tracking/haarcascade_frontalface_default.xml')

In [ ]:
!pip install yt-dlp

In [ ]:
from yt_dlp import YoutubeDL

In [ ]:
#Загрузка трансляции
url = "https://rutube.ru/video/6bed3b96454ff7d87583f3969ebb5f0f/"

ydl_opts = {
    'format': 'best', #выбор лучшего доступного формата
    'quiet':True,
}
with YoutubeDL(ydl_opts) as ydl:
    info=ydl.extract_info(url,download=False)
    video_url=info['url']


In [1]:
#Загрузка видео файла трансляции с rutube.ru

url = "https://rutube.ru/video/6bed3b96454ff7d87583f3969ebb5f0f/"

ydl_opts = {
    'format': 'best', #выбор лучшего доступного формата
    'outtmpl':'video.%(ext)s',
}

with YoutubeDL(ydl_opts) as ydl:
    ydl.download([url])
video_url='video.mp4'

ModuleNotFoundError: No module named 'yt_dlp'

In [ ]:
from google.colab.patches import cv2_imshow  #используем в колабе cv2_imshow вместо cv2.imshow

camera = cv2.VideoCapture(video_url)
#Число, которое мы передаем, означает источник. 0 – первая веб-камера в вашей системе, 1 – вторая и т.д.
#Если вы хотите загрузить уже существующий видеофайл вместо захвата прямой трансляции, просто на место номера вставьте путь к нему.
if not camera.isOpened():
    print('ошибка не удалось открыть видео поток')
else:
    #ret, frame = camera.read()
    #cv2_imshow('video feed', camera.frame)
    a=1
camera  #свойства   https://www.geeksforgeeks.org/how-to-get-properties-of-python-cv2-videocapture-object/
print("CV_CAP_PROP_FRAME_WIDTH: '{}'".format(camera.get(cv2.CAP_PROP_FRAME_WIDTH))) #ширина кадра в видео потоке в пикселях
print("CV_CAP_PROP_FRAME_HEIGHT : '{}'".format(camera.get(cv2.CAP_PROP_FRAME_HEIGHT)))
print("CAP_PROP_FPS : '{}'".format(camera.get(cv2.CAP_PROP_FPS))) #количество кадров в сек
print("CAP_PROP_POS_MSEC : '{}'".format(camera.get(cv2.CAP_PROP_POS_MSEC)))
print("CAP_PROP_FRAME_COUNT  : '{}'".format(camera.get(cv2.CAP_PROP_FRAME_COUNT)))
print("CAP_PROP_BRIGHTNESS : '{}'".format(camera.get(cv2.CAP_PROP_BRIGHTNESS)))
print("CAP_PROP_CONTRAST : '{}'".format(camera.get(cv2.CAP_PROP_CONTRAST)))
print("CAP_PROP_SATURATION : '{}'".format(camera.get(cv2.CAP_PROP_SATURATION)))
print("CAP_PROP_HUE : '{}'".format(camera.get(cv2.CAP_PROP_HUE)))
print("CAP_PROP_GAIN  : '{}'".format(camera.get(cv2.CAP_PROP_GAIN)))
print("CAP_PROP_CONVERT_RGB : '{}'".format(camera.get(cv2.CAP_PROP_CONVERT_RGB)))


CV_CAP_PROP_FRAME_WIDTH: '1920.0'
CV_CAP_PROP_FRAME_HEIGHT : '1080.0'
CAP_PROP_FPS : '25.0'
CAP_PROP_POS_MSEC : '0.0'
CAP_PROP_FRAME_COUNT  : '25504.0'
CAP_PROP_BRIGHTNESS : '0.0'
CAP_PROP_CONTRAST : '0.0'
CAP_PROP_SATURATION : '0.0'
CAP_PROP_HUE : '0.0'
CAP_PROP_GAIN  : '0.0'
CAP_PROP_CONVERT_RGB : '1.0'


In [2]:

kcf_tracker = None  # объект трекера инициализируется при первой детекции лица

#Затем создадим цикл while, чтобы получать из источника кадр за кадром.
while(1):
    ret, frame = camera.read()
    frame = cv2.flip(frame, 1)

    print('считывание кадра',ret)   #Функция cap.read() возвращает логическое значение (True/False) и кадр. Если кадр был считан верно, то возвращается True.
    if ret==True:
        cv2_imshow(frame)


    kcf_tracker_box = None  # результат работы трекера

    if kcf_tracker is not None:
        # обновляем трекер и получаем результат трекинга
        ok, box = kcf_tracker.update(frame)
        # сохраняем результат трекинга
        if ok:
            kcf_tracker_box = box

    # преобразуем изображение в чернобелый формат
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # запускаем детектор лиц
    faces = face_cascade.detectMultiScale(gray, 1.3, 3)

    # инициализируем трекер первой детекцией
    if len(faces) != 0 and kcf_tracker is None:
        kcf_tracker = cv2.TrackerKCF_create()
        (x, y, w, h) = faces[0]
        kcf_tracker.init(frame, (x,y,w,h))

    for (x,y,w,h) in faces:
        # отрисовываем детекцию лиц
        cv2.rectangle(frame,(x,y),(x+w,y+h),(255,0,0), 2)
        cv2.putText(frame, 'Detector', (x, y - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 1,
                    (255, 0, 0), 2, cv2.LINE_AA)

    # отрисовываем результат трекера
    if kcf_tracker_box is not None:
        (x, y, w, h) = map(int, kcf_tracker_box)
        cv2.rectangle(frame, (x,y),(x+w,y+h), (0, 0, 255), 2)
        cv2.putText(frame, 'Tracker', (x, y - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 1,
                    (0, 0, 255), 2, cv2.LINE_AA)

    cv2_imshow(frame)
    interrupt=cv2.waitKey(10)

    # выход по нажатию на клавишу 'q'
    if  interrupt & 0xFF == ord('q'):
        break



"kcf_tracker = None  # объект трекера инициализируется при первой детекции лица\n\n#Затем создадим цикл while, чтобы получать из источника кадр за кадром.\nwhile(1):\n    ret, frame = camera.read()\n    frame = cv2.flip(frame, 1)\n\n    print('считывание кадра',ret)   #Функция cap.read() возвращает логическое значение (True/False) и кадр. Если кадр был считан верно, то возвращается True.\n    if ret==True:\n        cv2_imshow(frame)\n\n\n    kcf_tracker_box = None  # результат работы трекера\n\n    if kcf_tracker is not None:\n        # обновляем трекер и получаем результат трекинга\n        ok, box = kcf_tracker.update(frame)\n        # сохраняем результат трекинга\n        if ok:\n            kcf_tracker_box = box\n\n    # преобразуем изображение в чернобелый формат\n    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)\n\n    # запускаем детектор лиц\n    faces = face_cascade.detectMultiScale(gray, 1.3, 3)\n\n    # инициализируем трекер первой детекцией\n    if len(faces) != 0 and kcf_

In [ ]:
#После того, как мы закончим использовать камеру, ее нужно «освободить». Если мы этого не сделаем, то в следующий раз при попытке ее использовать, вы получите ошибку.
camera.release()
cv2.destroyAllWindows()